In [18]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
from datetime import datetime
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
start = time.time()


In [19]:
# read CSV and create Pandas dataframe
df = pd.read_csv("trip_data.csv",
                index_col="request_datetime",
                infer_datetime_format=True,
                parse_dates=True)

# create a numerical index
df.reset_index(inplace=True)

# preview df
display(df)
display(df.dtypes)
display(df.columns)
display(len(df))

,request_datetime,hvfhs_license_num,dispatching_base_num,originating_base_num,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,2023-01-01 01:23:54,HV0003,B03404,B03404,2023-01-01 01:32:20,2023-01-01 01:34:04,2023-01-01 01:45:04,235,18,2.060,...,2.12,0.00,0.0,0.0,14.02,N,N,,N,N
1,2023-01-01 01:58:19,HV0003,B03404,B03404,2023-01-01 02:02:31,2023-01-01 02:02:53,2023-01-01 02:25:52,37,33,4.380,...,3.70,0.00,0.0,0.0,33.24,N,N,,N,N
2,2023-01-01 01:59:09,HV0003,B03404,B03404,2023-01-01 02:02:30,2023-01-01 02:04:02,2023-01-01 02:13:52,232,148,1.390,...,3.02,2.75,0.0,0.0,45.18,N,N,,N,N
3,2023-01-01 02:14:02,HV0003,B03404,B03404,2023-01-01 02:22:59,2023-01-01 02:23:19,2023-01-01 02:31:00,255,256,1.240,...,3.17,0.00,0.0,0.0,31.67,N,N,,N,N
4,2023-01-01 02:16:53,HV0003,B03404,B03404,2023-01-01 02:23:39,2023-01-01 02:24:02,2023-01-01 02:41:11,223,129,2.760,...,2.46,0.00,0.0,0.0,22.54,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2023-12-31 22:08:52,HV0005,B03406,NaN,NaN,2023-12-31 22:20:42,2023-12-31 22:43:17,60,244,4.694,...,5.25,0.00,0.0,0.0,20.30,N,N,N,N,N
11996,2023-12-31 22:17:11,HV0005,B03406,NaN,NaN,2023-12-31 22:23:08,2023-12-31 22:44:52,188,165,4.222,...,3.82,0.00,0.0,0.0,17.96,N,N,N,N,N
11997,2023-12-31 22:30:20,HV0003,B03404,B03404,2023-12-31 22:34:13,2023-12-31 22:36:14,2023-12-31 22:59:56,225,148,5.470,...,2.25,2.75,0.0,0.0,20.55,N,N,N,N,N
11998,2023-12-31 22:31:17,HV0003,B03404,B03404,2023-12-31 22:33:22,2023-12-31 22:35:24,2023-12-31 23:23:07,65,50,6.140,...,3.05,2.75,0.0,0.0,39.12,N,N,N,N,Y


request_datetime        datetime64[ns]
hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
on_scene_datetime               object
pickup_datetime                 object
dropoff_datetime                object
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

Index(['request_datetime', 'hvfhs_license_num', 'dispatching_base_num',
       'originating_base_num', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

12000

Arrange and filter dataframe.

In [20]:
# Renaming columns
column_dict = {
    'hvfhs_license_num': 'rideshare_company',
    'dispatching_base_num': 'd1',
    'originating_base_num': 'd2',
    'request_datetime': 'request_time',
    'on_scene_datetime': 'd8',
    'pickup_datetime': 'd9',
    'dropoff_datetime': 'd10',
    'PULocationID': 'pickup_zone',
    'DOLocationID': 'dropoff_zone',
    'trip_miles': 'trip_length_miles',
    'trip_time': 'trip_time_seconds',
    'base_passenger_fare': 'base_passenger_fare_dollars',
    'tolls': 'tolls_dollars',
    'bcf': 'black_car_fund_dollars',
    'sales_tax': 'sales_tax_dollars',
    'congestion_surcharge': 'congestion_surcharge_dollars',
    'airport_fee': 'airport_fee_dollars',
    'tips': 'tip_dollars',
    'driver_pay': 'driver_pay_dollars',
    'shared_request_flag': 'd3',
    'shared_match_flag': 'd4',
    'access_a_ride_flag': 'd5',
    'wav_request_flag': 'd6',
    'wav_match_flag': 'd7',
    }

df.rename(columns=column_dict, inplace=True)

# adapted from: https://github.com/Aweymouth13/rideshare_analysis/blob/main/ETL_analysis.ipynb

# Drop columns
columns_dropout = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', "d8", "d9", "d10"]
df.drop(columns=columns_dropout, inplace=True)

# Move columns
move_column = df.pop("request_time")
df.insert(3, "request_time", move_column)

In [21]:
# create y-hat for good fares
from scipy import stats

# create total_pay column
df["driver_total_pay_dollars"] = df["driver_pay_dollars"] + df["tip_dollars"]
df["driver_total_pay_dollars"]

# set "good fare" as being > 2 z-score (<5% of total data)
df["pay_z"] = stats.zscore((df["driver_pay_dollars"]))
df["good_fare"] = df["pay_z"].where(df["pay_z"] > 2, 0).astype(bool)

# delete pay_z
df.drop(columns="pay_z", inplace=True)

# check
df["good_fare"].value_counts()

good_fare
False    11480
True       520
Name: count, dtype: int64

Datetime adjustments.

In [22]:
# convert datetime to string categories
# these will be kept as str() because they are categories
df["request_month"] = df["request_time"].dt.strftime("%b")
df["request_date"] = df["request_time"].dt.strftime("%d")
df["request_hour"] = df["request_time"].dt.strftime("%H")
df["request_minute"] = df["request_time"].dt.strftime("%M")
df["request_day_of_week"] = df["request_time"].dt.strftime("%a")

# function to determine time of day
def time_of_day(hour):
    if 6 <= int(hour) < 12:
        return "morning"
    elif 12 <= int(hour) < 18:
        return "afternoon"
    elif 18 <= int(hour) < 22:
        return "evening"
    else:
        return "night"
    
# record function results in new column (applied to existing column)
df["time_of_day"] = df["request_hour"].apply(time_of_day)

# drop dateetime columns because many ml models cannot work directly with this dtype
df.drop(columns="request_time", inplace=True)

Add str() location information and borough columns from taxi_zone_lookup.csv.

In [23]:
# access taxi zone csv
location_encoding_csv = pd.read_csv(Path("taxi_zone_lookup.csv"))

# create columns for boroughs and populate with borough names
# create a dictionary of boroughs based on LocationID
borough_id_dict = dict(zip(location_encoding_csv["LocationID"], location_encoding_csv["Borough"]))

# replace location id with borough columns
df["pickup_borough"] = df["pickup_zone"].replace(borough_id_dict)
df["dropoff_borough"] = df["pickup_zone"].replace(borough_id_dict)

# convert ID to zone text
# create a dictionary of locations based on LocationID
zone_id_dict = dict(zip(location_encoding_csv["LocationID"], location_encoding_csv["Zone"]))

# replace location ID with zone name
df["pickup_zone"].replace(zone_id_dict, inplace=True)
df["dropoff_zone"].replace(zone_id_dict, inplace=True)

Add str() company names from PDF.

In [24]:
# Convert "hvfhs_license_num" to company name (using data_dictionary_trip_records_hvfhs.pdf)
# create dictionary for companies, using PDF
companies_dict = {
    "HV0002": "Juno",
    "HV0003": "Uber",
    "HV0004": "Via",
    "HV0005": "Lyft"
}
# replace company code with company name
df["rideshare_company"].replace(companies_dict, inplace=True)

Create column for tip bool.

In [25]:
# copy tip column and set it as boolean datatype
# false is no tip
df["tip_or_no_tip"] = df["tip_dollars"].astype(bool)

df

,rideshare_company,pickup_zone,dropoff_zone,trip_length_miles,trip_time_seconds,base_passenger_fare_dollars,tolls_dollars,black_car_fund_dollars,sales_tax_dollars,congestion_surcharge_dollars,...,good_fare,request_month,request_date,request_hour,request_minute,request_day_of_week,time_of_day,pickup_borough,dropoff_borough,tip_or_no_tip
0,Uber,University Heights/Morris Heights,Bedford Park,2.060,660,23.87,0.00,0.72,2.12,0.00,...,False,Jan,01,01,23,Sun,night,Bronx,Bronx,False
1,Uber,Bushwick South,Brooklyn Heights,4.380,1379,41.66,0.00,1.25,3.70,0.00,...,False,Jan,01,01,58,Sun,night,Brooklyn,Brooklyn,False
2,Uber,Two Bridges/Seward Park,Lower East Side,1.390,590,34.03,0.00,1.02,3.02,2.75,...,False,Jan,01,01,59,Sun,night,Manhattan,Manhattan,False
3,Uber,Williamsburg (North Side),Williamsburg (South Side),1.240,461,35.71,0.00,1.07,3.17,0.00,...,False,Jan,01,02,14,Sun,night,Brooklyn,Brooklyn,False
4,Uber,Steinway,Jackson Heights,2.760,1029,27.69,0.00,0.83,2.46,0.00,...,False,Jan,01,02,16,Sun,night,Queens,Queens,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,Lyft,Crotona Park East,Washington Heights South,4.694,1355,57.42,0.78,1.63,5.25,0.00,...,False,Dec,31,22,08,Sun,night,Bronx,Bronx,False
11996,Lyft,Prospect-Lefferts Gardens,Midwood,4.222,1304,47.12,0.00,1.18,3.82,0.00,...,False,Dec,31,22,17,Sun,night,Brooklyn,Brooklyn,False
11997,Uber,Stuyvesant Heights,Lower East Side,5.470,1422,25.40,0.00,0.70,2.25,2.75,...,False,Dec,31,22,30,Sun,night,Brooklyn,Brooklyn,False
11998,Uber,Downtown Brooklyn/MetroTech,Clinton West,6.140,2863,34.31,0.00,0.94,3.05,2.75,...,False,Dec,31,22,31,Sun,night,Brooklyn,Brooklyn,False


Handling nulls.

In [26]:
# Check for null datasets
display(df.isnull().sum())

# drop all null values
df.dropna(inplace=True)

# check
df.isnull().sum().sum()

rideshare_company               0
pickup_zone                     0
dropoff_zone                    0
trip_length_miles               0
trip_time_seconds               0
base_passenger_fare_dollars     0
tolls_dollars                   0
black_car_fund_dollars          0
sales_tax_dollars               0
congestion_surcharge_dollars    0
airport_fee_dollars             0
tip_dollars                     0
driver_pay_dollars              0
driver_total_pay_dollars        0
good_fare                       0
request_month                   0
request_date                    0
request_hour                    0
request_minute                  0
request_day_of_week             0
time_of_day                     0
pickup_borough                  0
dropoff_borough                 0
tip_or_no_tip                   0
dtype: int64

0

In [27]:
display(df.head())
display(df.tail())

,rideshare_company,pickup_zone,dropoff_zone,trip_length_miles,trip_time_seconds,base_passenger_fare_dollars,tolls_dollars,black_car_fund_dollars,sales_tax_dollars,congestion_surcharge_dollars,...,good_fare,request_month,request_date,request_hour,request_minute,request_day_of_week,time_of_day,pickup_borough,dropoff_borough,tip_or_no_tip
0,Uber,University Heights/Morris Heights,Bedford Park,2.06,660,23.87,0.0,0.72,2.12,0.00,...,False,Jan,01,01,23,Sun,night,Bronx,Bronx,False
1,Uber,Bushwick South,Brooklyn Heights,4.38,1379,41.66,0.0,1.25,3.70,0.00,...,False,Jan,01,01,58,Sun,night,Brooklyn,Brooklyn,False
2,Uber,Two Bridges/Seward Park,Lower East Side,1.39,590,34.03,0.0,1.02,3.02,2.75,...,False,Jan,01,01,59,Sun,night,Manhattan,Manhattan,False
3,Uber,Williamsburg (North Side),Williamsburg (South Side),1.24,461,35.71,0.0,1.07,3.17,0.00,...,False,Jan,01,02,14,Sun,night,Brooklyn,Brooklyn,False
4,Uber,Steinway,Jackson Heights,2.76,1029,27.69,0.0,0.83,2.46,0.00,...,False,Jan,01,02,16,Sun,night,Queens,Queens,False


,rideshare_company,pickup_zone,dropoff_zone,trip_length_miles,trip_time_seconds,base_passenger_fare_dollars,tolls_dollars,black_car_fund_dollars,sales_tax_dollars,congestion_surcharge_dollars,...,good_fare,request_month,request_date,request_hour,request_minute,request_day_of_week,time_of_day,pickup_borough,dropoff_borough,tip_or_no_tip
11995,Lyft,Crotona Park East,Washington Heights South,4.694,1355,57.42,0.78,1.63,5.25,0.00,...,False,Dec,31,22,08,Sun,night,Bronx,Bronx,False
11996,Lyft,Prospect-Lefferts Gardens,Midwood,4.222,1304,47.12,0.00,1.18,3.82,0.00,...,False,Dec,31,22,17,Sun,night,Brooklyn,Brooklyn,False
11997,Uber,Stuyvesant Heights,Lower East Side,5.470,1422,25.40,0.00,0.70,2.25,2.75,...,False,Dec,31,22,30,Sun,night,Brooklyn,Brooklyn,False
11998,Uber,Downtown Brooklyn/MetroTech,Clinton West,6.140,2863,34.31,0.00,0.94,3.05,2.75,...,False,Dec,31,22,31,Sun,night,Brooklyn,Brooklyn,False
11999,Lyft,East Flatbush/Remsen Village,Erasmus,1.236,555,10.64,0.00,0.29,0.94,0.00,...,False,Dec,31,23,14,Sun,night,Brooklyn,Brooklyn,False


## Encode the dataset categorical variables using OneHotEncoder, and then place the encoded variables into a new DataFrame.

In [28]:
# Create a list of categorical variables
categorical_variables = list(df.dtypes[df.dtypes=='object'].index)

# Display the categorical variables list
categorical_variables

['rideshare_company',
 'pickup_zone',
 'dropoff_zone',
 'request_month',
 'request_date',
 'request_hour',
 'request_minute',
 'request_day_of_week',
 'time_of_day',
 'pickup_borough',
 'dropoff_borough']

In [29]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

In [30]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(df[categorical_variables])

In [31]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

,rideshare_company_Lyft,rideshare_company_Uber,pickup_zone_Allerton/Pelham Gardens,pickup_zone_Alphabet City,pickup_zone_Arden Heights,pickup_zone_Arrochar/Fort Wadsworth,pickup_zone_Astoria,pickup_zone_Auburndale,pickup_zone_Baisley Park,pickup_zone_Bath Beach,...,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,pickup_borough_Staten Island,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [32]:
# Encode the non categorical variables using OneHotEncoder
non_categorical_variables = df.drop(columns=categorical_variables)

# preview
non_categorical_variables.head()

,trip_length_miles,trip_time_seconds,base_passenger_fare_dollars,tolls_dollars,black_car_fund_dollars,sales_tax_dollars,congestion_surcharge_dollars,airport_fee_dollars,tip_dollars,driver_pay_dollars,driver_total_pay_dollars,good_fare,tip_or_no_tip
0,2.06,660,23.87,0.0,0.72,2.12,0.00,0.0,0.0,14.02,14.02,False,False
1,4.38,1379,41.66,0.0,1.25,3.70,0.00,0.0,0.0,33.24,33.24,False,False
2,1.39,590,34.03,0.0,1.02,3.02,2.75,0.0,0.0,45.18,45.18,False,False
3,1.24,461,35.71,0.0,1.07,3.17,0.00,0.0,0.0,31.67,31.67,False,False
4,2.76,1029,27.69,0.0,0.83,2.46,0.00,0.0,0.0,22.54,22.54,False,False


In [33]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([non_categorical_variables, encoded_df], axis=1)

# Review the DataFrame
display(encoded_df)

# Check for nulls
display(encoded_df.isna().sum().sum())

,trip_length_miles,trip_time_seconds,base_passenger_fare_dollars,tolls_dollars,black_car_fund_dollars,sales_tax_dollars,congestion_surcharge_dollars,airport_fee_dollars,tip_dollars,driver_pay_dollars,...,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,pickup_borough_Staten Island,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island
0,2.060,660,23.87,0.00,0.72,2.12,0.00,0.0,0.0,14.02,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,4.380,1379,41.66,0.00,1.25,3.70,0.00,0.0,0.0,33.24,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.390,590,34.03,0.00,1.02,3.02,2.75,0.0,0.0,45.18,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.240,461,35.71,0.00,1.07,3.17,0.00,0.0,0.0,31.67,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.760,1029,27.69,0.00,0.83,2.46,0.00,0.0,0.0,22.54,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,4.694,1355,57.42,0.78,1.63,5.25,0.00,0.0,0.0,20.30,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11996,4.222,1304,47.12,0.00,1.18,3.82,0.00,0.0,0.0,17.96,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
11997,5.470,1422,25.40,0.00,0.70,2.25,2.75,0.0,0.0,20.55,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
11998,6.140,2863,34.31,0.00,0.94,3.05,2.75,0.0,0.0,39.12,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


0

In [34]:
# export to csv
encoded_df.to_csv("encoded_data.csv")